In [ ]:
import torch
import numpy as np
import hyperspy.api as hs
import py4DSTEM
from torch.utils.data import Dataset

In [27]:
class STEMDATA(Dataset):
    """
    data_4d: (X, Y, Kx, Ky)
    Output: clip of shape (T, 1, Kx, Ky)
    """
    def __init__(self, filepath, T: int = 5):

        s = hs.load(filepath, reader='HSPY')
        datacube = py4DSTEM.DataCube(s.data)
        self.data_4d: np.ndarray = datacube.data
        self.D = self.data_4d
        self.X, self.Y, self.Kx, self.Ky = self.data_4d.shape
        self.T = T
        self.half = T // 2

        # flatten (x,y) -> z
        self.Dz = self.data_4d.reshape(self.X * self.Y, self.Kx, self.Ky)

        # valid centers
        self.valid_z = range(self.half, self.X * self.Y - self.half)

    def __len__(self):
        return len(self.valid_z)

    def __getitem__(self, idx):
        z = self.valid_z[idx]
        clip_np = self.Dz[z - self.half : z + self.half + 1]  # (T, Kx, Ky)
        clip = torch.from_numpy(clip_np).float().unsqueeze(1)  # (T,1,Kx,Ky)
        return clip, z


In [31]:
filepath = r"../../data/raw/4D-STEM_data_for_theophylline/20220711_182642_data_150kX_binned2.hdf5"
ds = STEMDATA(filepath, T=5)

WARNING | Hyperspy | This file contains a signal provided by the pyxem Python package that is not currently installed. The signal will be loaded into a generic HyperSpy signal. Consider installing pyxem to load this dataset into its original signal class. (hyperspy.io:801)
WARNING | Hyperspy | `signal_type='electron_diffraction'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals. (hyperspy.io:745)


Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.


In [40]:
from ..scripts.eval import load_model
model, optimizer, args = load_model(PATH, parallel=parallel, pretrained=pretrained, old=old, load_opt=load_opt)


model = model.to(device).eval()
with torch.no_grad():
    for clip, z in dl:
        clip = clip.to(device)              # (B,5,1,257,257)


ImportError: attempted relative import with no known parent package